In [88]:
import wget
import calendar
import ujson
import gzip
import matplotlib
import pandas
import os
import re
import time
import pprint
import importlib
## functions
import github_parsing_functions as ghp

In [89]:
def parse_pull_request_events(json_data):
    result = {'event': 'PullRequestEvent'}
    result['repo_id'] = json_data['payload']['pull_request']['base']['repo']['id'] 
    result['repo_name'] = json_data['payload']['pull_request']['base']['repo']['name']
    result['description'] = json_data['payload']['pull_request']['base']['repo']['description']
    result['language'] = json_data['payload']['pull_request']['base']['repo']['language']
    result['stargazers_count'] = json_data['payload']['pull_request']['base']['repo']['stargazers_count']
    result['created_at'] = json_data['created_at']
    result['actor_id'] = json_data['actor']['id']
    result['actor_login'] = json_data['actor']['login']
    result['org'] = False
    result['org_id'] = None
    if 'org' in json_data:
        result['org'] = True
        result['org_id'] = json_data['org']['id']
    return result

def parse_pull_request_review_comment_events(json_data):
    result = {'event': 'PullRequestReviewCommentEvent'}
    result['language'] = json_data['payload']['pull_request']['base']['repo']['language'] 
    result["stargazers_count"] = json_data['payload']['pull_request']['base']['repo']["stargazers_count"] 
    result['repo_id'] = json_data['payload']['pull_request']['base']['repo']['id'] 
    result['repo_name'] = json_data['payload']['pull_request']['base']['repo']['full_name'] 
    result['created_at'] = json_data['created_at']
    result['actor_id'] = json_data['actor']['id']
    result['actor_login'] = json_data['actor']['login']
    result['org'] = False
    result['org_id'] = None
    if 'org' in json_data:
        result['org'] = True
        result['org_id'] = json_data['org']['id']
    return result

def parse_fork_event(json):
    result = {}
    result['event'] = 'ForkEvent'
    result['repo_id'] = json['repo']['id']
    result['repo_name'] = json['repo']['name']
    result['actor_id'] = json['actor']['id']
    result['actor_login'] = json['actor']['login']
    result['org'] = False
    result['org_id'] = None
    result['created_at'] = json['created_at']
    result['language'] = json['payload']['forkee']['language']
    result['stargazers_count'] = json['payload']['forkee']['stargazers_count']
    if 'org' in json:
        result['org'] = json['org']['id']
        result['org_id'] = json['org']['id']     
    return result

def parse_watch_event(json_data):
    result = {}
    result['created_at'] = json_data["created_at"]
    result['event'] = "WatchEvent"
    result['repo_id'] = json_data["repo"]["id"]
    result['repo_name'] = json_data["repo"]["name"]
    result['action'] = json_data["payload"]["action"]
    result['actor_id'] = json_data["actor"]["id"]
    result['actor_login'] = json_data["actor"]["login"]
    result['org'] = False
    result['org_id'] =  None
    if 'org' in json_data:
        result['org'] = True
        result['org_id'] = json_data["org"]["id"]
    return result

### Download github archive files

In [5]:
url= 'http://data.githubarchive.org/2016-02-01-15.json.gz'
filename = wget.download(url)
filename

'2016-02-01-15.json (3).gz'

### Parse out events

In [90]:
fork_event = []
watch_event = []
pr_event = []
pr_reviewcomment_event = []

In [6]:

with gzip.open('2016-02-01-15.json.gz') as f:   
    for i, line in enumerate(f):
        json_data = ujson.loads(line)
        if (json_data["type"] == "ForkEvent"):
            fork_event.append(parse_fork_event(json_data))
        elif (json_data["type"] == "WatchEvent"):
            watch_event.append(parse_watch_event(json_data))
        elif (json_data["type"] == "PullRequestEvent"):
            pr_event.append(parse_pull_request_events(json_data))
        elif (json_data["type"] == "PullRequestReviewCommentEvent"):
            pr_reviewcomment_event.append(parse_pull_request_review_comment_events(json_data))
    f.close()

In [92]:
for hour in range(0, 24):
    with gzip.open('2016-02-07-'+str(hour)+'.json.gz') as f:   
        for i, line in enumerate(f):
            json_data = ujson.loads(line)
            if (json_data["type"] == "ForkEvent"):
                fork_event.append(parse_fork_event(json_data))
            elif (json_data["type"] == "WatchEvent"):
                watch_event.append(parse_watch_event(json_data))
            elif (json_data["type"] == "PullRequestEvent"):
                pr_event.append(parse_pull_request_events(json_data))
            elif (json_data["type"] == "PullRequestReviewCommentEvent"):
                pr_reviewcomment_event.append(parse_pull_request_review_comment_events(json_data))
        f.close()

In [93]:
watch_df = pandas.DataFrame.from_dict(watch_event, orient='columns', dtype=None)
fork_df = pandas.DataFrame.from_dict(fork_event, orient='columns', dtype=None)
pr_df = pandas.DataFrame.from_dict(pr_event, orient='columns', dtype=None)
pr_rc_df = pandas.DataFrame.from_dict(pr_reviewcomment_event, orient='columns', dtype=None)

In [94]:
fork_df.head()

,actor_id,actor_login,created_at,event,language,org,org_id,repo_id,repo_name,stargazers_count
0,147788,rapzo,2016-02-07T00:00:02Z,ForkEvent,None,3284117,3284117,8474328,karma-runner/karma-coverage,0
1,8035580,jfaquinojr,2016-02-07T00:00:16Z,ForkEvent,None,3195020,3195020,12004332,ligershark/side-waffle,0
2,16857734,n1cfury,2016-02-07T00:00:24Z,ForkEvent,Python,False,NaN,49541658,theabraxas/IT-Infrastructure-Super-Utility,0
3,2496411,dimitarsp,2016-02-07T00:00:27Z,ForkEvent,None,False,NaN,20274542,pitiwazou/Scripts-Blender,0
4,11396139,migueldf,2016-02-07T00:00:28Z,ForkEvent,None,13573168,13573168,40633703,jhu-ep-coursera/fullstack-course1-module3,0


In [95]:
frames = [pr_df, pr_rc_df, fork_df]
combined = pandas.concat(frames)
combined.head()

,actor_id,actor_login,created_at,description,event,language,org,org_id,repo_id,repo_name,stargazers_count
0,622699,harshavardhana,2016-02-07T00:00:03Z,Minio Browser,PullRequestEvent,JavaScript,True,695951,45657978,miniobrowser,6
1,1977052,corey-hammerton,2016-02-07T00:00:04Z,Web frontend for PuppetDB,PullRequestEvent,Python,True,8693967,11746441,puppetboard,409
2,13547129,Excoh,2016-02-07T00:00:05Z,Test GitHub and its functions.,PullRequestEvent,None,False,NaN,51227866,hello-world,0
3,622699,harshavardhana,2016-02-07T00:00:07Z,Minio Browser,PullRequestEvent,JavaScript,True,695951,45657978,miniobrowser,6
4,2207561,samchrisinger,2016-02-07T00:00:08Z,,PullRequestEvent,JavaScript,True,3344584,50667950,exp-addons,0


### Munge and reformat Repo information 

In [96]:
latest_by_repoid = combined.sort('created_at', ascending= False).drop_duplicates(subset="repo_id")
print(len(latest_by_repoid))
latest_by_repoid['description'] = latest_by_repoid['description'].fillna('') # get rid of nans
latest_by_repoid['org_id'] = latest_by_repoid['org_id'].fillna(0) # org_id of 0 means no org associated

repo_info = latest_by_repoid[['repo_id', 'repo_name', 'description', 'language', 'stargazers_count', 'org_id']]

18316


/Users/sperrochsy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [97]:
repo_info.tail()

,repo_id,repo_name,description,language,stargazers_count,org_id
9,38230607,lanterna,Java library for creating text-based GUIs,Java,202,0
8,23524933,ansible-ferm,Manage iptables firewall using ferm,Shell,14,8523739
7,48774945,vader,Codename for new Node.js/Express UPTV website,JavaScript,0,0
2,51227866,hello-world,Test GitHub and its functions.,None,0,0
0,8474328,karma-runner/karma-coverage,,None,0,3284117


### Insert Rows into DB

In [84]:
import psycopg2
 
import sys
 
db_con = None

In [85]:
db_con = psycopg2.connect(database='halcyon', user='postgres', password='hi')
cursor = db_con.cursor()

#### Upsert Repository information (insert all rows as single transaction)

In [86]:
for index, row in repo_info.iterrows():
    
    sql_str = """INSERT INTO halcyon."Test_Repos" ( id, name, description, language, num_stars, org) 
                 VALUES (%s, %s, %s, %s, %s, %s)
                 ON CONFLICT (id) DO UPDATE SET 
                 name=excluded.name, description=excluded.description,
                 language=excluded.language, num_stars=excluded.num_stars;"""
        
    cursor.execute(sql_str, (row['repo_id'], row['repo_name'], row['description'], row['language'], row['stargazers_count'], row['org_id']))
db_con.commit()

#### Clean up Watch Events and put them into DB

In [98]:
watch_df.head()

,action,actor_id,actor_login,created_at,event,org,org_id,repo_id,repo_name
0,started,2054138,hyrmedia,2016-02-07T00:00:03Z,WatchEvent,False,NaN,19133534,EvanOxfeld/angular-selectize.js
1,started,2086265,ovgu12,2016-02-07T00:00:05Z,WatchEvent,True,2185347,519390,emmetio/emmet
2,started,1407119,ymattw,2016-02-07T00:00:05Z,WatchEvent,True,647025,1446474,pypa/virtualenv
3,started,1223190,danielvestergaard,2016-02-07T00:00:06Z,WatchEvent,True,57059,9157414,scala/async
4,started,142254,bloudermilk,2016-02-07T00:00:08Z,WatchEvent,False,NaN,25670446,tomchentw/react-google-maps


In [99]:
watch_df['date'] = watch_df['created_at'].map(lambda date_str: date_str.split('T')[0])
watch_df['hour'] = watch_df['created_at'].map(lambda date_str: date_str.split('T')[1].split(':')[0])
#watch_df['hour'] = watch_df['created_at'].split('T')[1].split(':')[0]

In [100]:
watch_df.head()

,action,actor_id,actor_login,created_at,event,org,org_id,repo_id,repo_name,date,hour
0,started,2054138,hyrmedia,2016-02-07T00:00:03Z,WatchEvent,False,NaN,19133534,EvanOxfeld/angular-selectize.js,2016-02-07,00
1,started,2086265,ovgu12,2016-02-07T00:00:05Z,WatchEvent,True,2185347,519390,emmetio/emmet,2016-02-07,00
2,started,1407119,ymattw,2016-02-07T00:00:05Z,WatchEvent,True,647025,1446474,pypa/virtualenv,2016-02-07,00
3,started,1223190,danielvestergaard,2016-02-07T00:00:06Z,WatchEvent,True,57059,9157414,scala/async,2016-02-07,00
4,started,142254,bloudermilk,2016-02-07T00:00:08Z,WatchEvent,False,NaN,25670446,tomchentw/react-google-maps,2016-02-07,00


### Group Watches and insert into DB

In [103]:
grouped_watches = pandas.core.frame.DataFrame({'count' : watch_df.groupby(['repo_id', 'repo_name', 'date']).size().order(ascending=False)}).reset_index()

/Users/sperrochsy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


In [105]:
grouped_watches.head(20)

,repo_id,repo_name,date,count
0,50301368,p-e-w/maybe,2016-02-07,2059
1,51184395,jayphelps/git-blame-someone-else,2016-02-07,675
2,48855539,oduslabs/Console-Kit,2016-02-07,602
3,28452637,kennethreitz/records,2016-02-07,558
4,41509732,zymtom/DidIStealThis,2016-02-07,511
5,28457823,FreeCodeCamp/FreeCodeCamp,2016-02-07,470
6,35699987,Hotsotse/Hang-man,2016-02-07,419
7,50264296,jgthms/bulma,2016-02-07,310
8,9149146,pudo/dataset,2016-02-07,293
9,51025772,AriaFallah/WebpackTutorial,2016-02-07,288


In [77]:
for index, row in grouped_watches.iterrows():
    
    sql_str = """INSERT INTO halcyon."Test_Hourly_Watches" ( repo_id, repo_name, star_count, date, hour) 
                 VALUES (%s, %s, %s, %s, %s);"""
        
    cursor.execute(sql_str, (row['repo_id'], row['repo_name'], row['count'], row['date'], row['hour']))
db_con.commit()

### Combine Watches with Repo info to get aggregate stats

In [106]:
watch_lang = pandas.merge(left=watch_df,right=repo_lang, how='left', left_on='repo_id', right_on='repo_id')

In [107]:
watch_lang['filled_language'] = watch_lang['language'].fillna('No Language')
watch_lang.head()

,action,actor_id,actor_login,created_at,event,org,org_id,repo_id,repo_name,date,hour,language,filled_language
0,started,2054138,hyrmedia,2016-02-07T00:00:03Z,WatchEvent,False,NaN,19133534,EvanOxfeld/angular-selectize.js,2016-02-07,00,NaN,No Language
1,started,2086265,ovgu12,2016-02-07T00:00:05Z,WatchEvent,True,2185347,519390,emmetio/emmet,2016-02-07,00,NaN,No Language
2,started,1407119,ymattw,2016-02-07T00:00:05Z,WatchEvent,True,647025,1446474,pypa/virtualenv,2016-02-07,00,NaN,No Language
3,started,1223190,danielvestergaard,2016-02-07T00:00:06Z,WatchEvent,True,57059,9157414,scala/async,2016-02-07,00,NaN,No Language
4,started,142254,bloudermilk,2016-02-07T00:00:08Z,WatchEvent,False,NaN,25670446,tomchentw/react-google-maps,2016-02-07,00,NaN,No Language


In [110]:
watch_lang[(watch_lang['repo_id'] == 50301368)]

,action,actor_id,actor_login,created_at,event,org,org_id,repo_id,repo_name,date,hour,language,filled_language
52,started,5866758,almightyyeh,2016-02-07T00:01:24Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language
72,started,292969,kenlimmj,2016-02-07T00:02:12Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language
111,started,48586,galvao,2016-02-07T00:03:57Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language
141,started,48586,galvao,2016-02-07T00:04:41Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language
142,started,4390935,maddouri,2016-02-07T00:04:41Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language
177,started,9091144,james9909,2016-02-07T00:06:27Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language
179,started,234136,Braynid,2016-02-07T00:06:35Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language
259,started,1509413,gokaygurcan,2016-02-07T00:09:38Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language
478,started,1523594,jvperrin,2016-02-07T00:14:32Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language
507,started,4040623,mseri,2016-02-07T00:15:28Z,WatchEvent,False,NaN,50301368,p-e-w/maybe,2016-02-07,00,NaN,No Language


In [108]:
watch_lang.groupby(['repo_id', 'repo_name','filled_language', 'date', 'hour']).size().order(ascending=False).head()

/Users/sperrochsy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


repo_id   repo_name                   filled_language  date        hour
35699987  Hotsotse/Hang-man           No Language      2016-02-07  02      410
48855539  oduslabs/Console-Kit        No Language      2016-02-07  18      408
41509732  zymtom/DidIStealThis        No Language      2016-02-07  14      354
51215534  Lloydaaa/Googl-Library-PHP  No Language      2016-02-07  00      175
48855539  oduslabs/Console-Kit        No Language      2016-02-07  19      168
dtype: int64

In [87]:
repo_info.groupby

,repo_id,repo_name,description,language,stargazers_count,org_id
1307,48862284,learn-co-students/sinatra-integrating-models-w...,,None,0,8825476
1077,26295345,dotnet/corefx,,C#,9054,9141961
1306,40944209,mgwidmann/scrivener_html,,None,0,0
2960,50800317,ios-candidate-assignment,iOS Candidate Assignment,None,0,0
2961,30210041,BotBits,An Everybody Edits framework designed for perf...,C#,8,0
1305,48904888,Twentysix26/Red-DiscordBot,,None,0,0
2959,50839889,WPAppTest,,PHP,0,0
1304,6050093,guybedford/require-less,,None,0,0
2958,16844474,clappr,:clapper: An extensible media player for the web,JavaScript,2674,10685859
1076,15802645,withassociates/slices,,Ruby,32,13455
